In [ ]:
%pip install -r ../requirements.txt

## Load libraries

In [ ]:
from matplotlib import pyplot as plt  # noqa: E402
from omegaconf import OmegaConf  # noqa: E402

%matplotlib inline

from modules import config, utils  # noqa: E402

## Visualize dataset

In [ ]:
def view_image(ds):
    image = next(ds)  # extract 1 batch from the dataset
    image = image.numpy()

    fig = plt.figure(figsize=(22, 22))
    for i in range(20):
        ax = fig.add_subplot(4, 5, i + 1, xticks=[], yticks=[])
        ax.imshow(image[i])

In [ ]:
# Data config path
data_config_path = "../config/config.yaml"
cfg = OmegaConf.load(data_config_path)
load_confg = OmegaConf.to_container(cfg)
conf = config.LoadConfig(**load_confg)

In [ ]:
# Prepare dataloaders
dataset_train_class = utils.DummyDataset(train=True, dtype=conf.train.dtype, config=conf.data)
loader_train = utils.DataLoader(dataset_train_class, distributed=False)

## View

In [ ]:
view_image(loader_train())